In [ ]:
from utils import propose_region, bet_unet
from model import load_model, unet
from tqdm import tqdm

import numpy as np
import nibabel as nib

import cv2
import os
import time

In [ ]:
def detect_file(filename, model, unet):
    data = nib.load(filename)
    mask = nib.load(filename)
    mask_roi = nib.load(filename)

    width, height, frame_num = data.shape
    matrix = data.get_data()
    bet_matrix = np.uint8(bet_unet(matrix, unet))

    start = time.time()
    for i in tqdm(range(frame_num), desc='Detect in {}'.format(os.path.basename(filename))):
        img = matrix[:, :, i]
        bet = bet_matrix[:, :, i]
        selected, mask_ROI = detect(img, model, bet)
        mask.get_data()[:, :, i] = selected
        mask_roi.get_data()[:, :, i] = mask_ROI
    end = time.time()

    print('Using time {}s'.format(end - start))
    return mask, mask_roi


def detect(img, model, bet, pad=10, is_debug=False):
    width, height = img.shape
    mask_ROI = propose_region(img, is_debug) & bet
    num, labels, stats, centroid = cv2.connectedComponentsWithStats(
        mask_ROI, connectivity=8)
    selected = np.zeros_like(img)
    for i, stat, center in zip(range(num), stats, centroid):
        if is_debug:
            print(i, stat, center)
        x, y, w, h, area = stat
        # remove background
        if x == 0 and y == 0:
            continue
        cx, cy = np.uint8(center)
        # crop a slice around pickle
        # valid bounder
        if cx - pad < 0 or cx + pad > height or cy - pad < 0 or cy + pad > width:
            continue
        if is_debug:
            print('label:{}, center:({},{})'.format(i, cx, cy))
        slice_img = img[cy - pad: cy + pad, cx - pad: cx + pad]
        slice_img = slice_img[:, :, np.newaxis]
        slice_img = np.expand_dims(slice_img, axis=0)
        res = model.predict(slice_img)
        if np.argmax(res) == 1:
            selected[labels == i] = np.max(res)
            if is_debug:
                print('label:{}, center:({},{})'.format(i, cx, cy))
    return selected, mask_ROI

路径定义，数据、模型与结果在example文件夹中

In [ ]:
data_root = 'example/data'
model_path = 'example/model'
result_path = 'example/result'

# 单张图片检测

In [ ]:
data_filename = '0507_90day50u_20190812_mouse51SWI.nii'
data_path = os.path.join(data_root, data_filename)

result_filename = '{}_detect.nii'.format(data_filename.split('/')[-1].split('.')[-2])

result_path = os.path.join(result_path, result_filename)

model_name = 'CNN_p10_e2000.h5'
unet_path = os.path.join(model_path, 'unet_BET2.hdf5')
clf_path = os.path.join(model_path, model_name)
bet_net = unet(pretrained_weights=unet_path)
model = load_model(clf_path)

mask, mask_roi = detect_file(data_path, model, bet_net)

nib.save(mask, result_path)
nib.save(mask_roi, 'roi.nii')

# 多张图片检测

In [ ]:
model_name = 'CNN_p10_e2000.h5'
clf_path = os.path.join(model_path, model_name)
model = load_model(clf_path)

input_path = os.path.join(data_root, './')

method = ['bet', 'heatmap', 'unet'] 分别对应形态学、Heatmap和Unet三种检测方法

In [ ]:
method = args.method
print('Using {} for detection...'.format(method))
if method == 'bet':
    method_path = os.path.join(model_path, 'unet_BET2.hdf5')
    from detect import detect_file
elif method == 'heatmap':
    method_path = os.path.join(model_path, 'unet_pm25_yuzq.hdf5')
    from detect_by_heatmap import detect_file
elif method == 'unet':
    method_path = os.path.join(model_path, 'unet_pm25_yuzq.hdf5')
    from detect_by_unet import detect_file
method_net = unet(pretrained_weights=method_path)

In [ ]:
NII_fileList = []
for dirName, subdirList, fileList in os.walk(input_path):
    for filename in fileList:
        if ".nii" in filename.lower():
            # print filename
            NII_fileList.append(os.path.join(dirName, filename))

for file in tqdm(NII_fileList, desc='Detect in {}'.format(os.path.abspath(input_path))):
    filename = os.path.basename(file)
    output_name = '{}_detect_by_{}.nii'.format(filename.split('.')[-2], method)
    output_path = os.path.join(result_path, output_name)
    mask, _ = detect_file(file, model, method_net)
    nib.save(mask, output_path)